In [ ]:
#!pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np        
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
df = pd.read_excel("online_retail_II.xlsx")

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(525461, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10

In [ ]:
post = df[df["StockCode"]=="POST"].index
df.drop(post,inplace=True)

In [ ]:
df = df.dropna()


In [ ]:
cancel = df[df["Invoice"]=="C"].index
df.drop(cancel,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df = df[df["Price"] > 0]

In [ ]:
df.shape

(416681, 8)

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
df_ger = df[df['Country'] == "Germany"]

In [ ]:
ger_inv_pro_df = create_invoice_product_df(df_ger, id=True)

In [ ]:
ger_inv_pro_df.head()

StockCode,10002,10125,10135,11001,15034,15036,15039,16012,16016,16033,16045,16052,16236,16237,16238,16259,17003,20617,20619,20622,20652,20653,20654,20658,20661,20665,20668,20671,20672,20674,20675,20676,20677,20679,20681,20682,20684,20685,20689,20692,20696,20702,20703,20711,20712,20713,20716,20717,20718,20719,20723,20724,20725,20726,20727,20728,20729,20730,20734,20738,20749,20750,20751,20752,20754,20755,20756,20758,20759,20760,20761,20762,20764,20766,20767,20768,20769,20770,20771,20772,20794,20798,20799,20816,20828,20829,20857,20864,20866,20867,20868,20871,20886,20914,20957,20961,20963,20964,20966,20967,20971,20972,20973,20974,20975,20977,20978,20979,20981,20982,20983,20984,20985,20990,20997,20998,21000,21001,21002,21011,21025,21026,21027,21033,21035,21039,21041,21042,21051,21056,21058,21059,21064,21067,21069,21070,21077,21078,21080,21082,21084,21085,21086,21088,21090,21091,21092,21094,21096,21098,21099,21100,21106,21108,21110,21114,21115,21116,21117,21121,21122,21124,21125,21126,21127,21130,21135,21136,21137,21144,21153,21154,21155,21156,21157,21162,21163,21164,21165,21166,21179,21186,21188,21190,21191,21197,21202,21203,21204,21205,21206,21207,21210,21211,21212,21213,21215,21216,21217,21218,21219,21220,21221,21222,21223,21224,21226,21231,21232,21238,21239,21240,21241,21242,21243,21244,21245,21246,21247,21248,21249,21251,21258,21259,21260,21263,21269,21270,21272,21275,21276,21278,21281,21282,21284,21285,21286,21291,21307,21319,21320,21324,21327,21328,21329,21330,21348,21354,21355,21357,21358,21361,21374,21380,21381,21383,21392,21393,21394,21395,21396,21397,21398,21399,21401,21410,21411,21414,21415,21416,21424,21426,21428,21429,21430,21431,21432,21439,21440,21445,21446,21447,21452,21461,21464,21467,21471,21472,21474,21479,21481,21482,21484,21485,21486,21488,21491,21494,21495,21496,21497,21498,21499,21500,21503,21506,21507,21508,21509,21519,21523,21524,21525,21527,21528,21529,21531,21533,21534,21535,21537,21539,21541,21542,21544,21555,21556,21558,21559,21561,21562,21563,21564,21576,21577,21578,21579,21580,21581,21586,21588,21590,21592,21609,21610,21611,21612,21614,21615,21618,21619,21620,21621,21622,21623,21625,21637,21640,21645,21648,21651,21652,21653,21655,21658,21661,21662,21667,21668,21669,21670,21671,21672,21673,21674,21675,21679,21680,21687,21690,21691,21693,21696,21697,21700,21703,21704,21705,21706,21707,21708,21709,21714,21715,21716,21717,21718,21719,21720,21721,21722,21723,21724,21725,21730,21731,21733,21735,21739,21741,21742,21746,21749,21752,21754,21755,21756,21763,21768,21775,21785,21786,21787,21788,21789,21790,21791,21801,21802,21803,21817,21827,21829,21830,21832,21833,21836,21837,21842,21843,21844,21845,21866,21870,21871,21872,21873,21875,21877,21878,21880,21881,21882,21883,21884,21889,21890,21891,21892,21894,21896,21903,21906,21913,21914,21915,21916,21917,21918,21922,21923,21925,21928,21929,21930,21931,21932,21933,21935,21936,21937,21942,21943,21944,21945,21946,21947,21949,21967,21972,21973,21974,21975,21976,21977,21980,21981,21982,21983,21984,21986,21987,21988,21989,21990,21992,21993,22021,22023,22024,22025,22027,22028,22029,22030,22031,22032,22033,22034,22035,22036,22037,22038,22040,22042,22043,22044,22045,22046,22047,22049,22052,22055,22058,22059,22062,22063,22064,22067,22068,22069,22070,22071,22072,22073,22074,22075,22076,22077,22078,22079,22080,22081,22082,22083,22084,22086,22088,22089,22090,22094,22099,22107,22108,22110,22111,22113,22114,22116,22117,22122,22124,22127,22128,22129,22130,22131,22132,22133,22134,22135,22136,22137,22138,22139,22141,22144,22145,22149,22150,22155,22168,22169,22170,22171,22173,22175,22176,22178,22179,22180,22181,22189,22191,22192,22193,22194,22197,22198,22199,22200,22201,22202,22203,22204,22207,22211,22212,22215,22218,22219,22221,22228,22229,22230,22231,22232,22233,22236,22242,22243,22244,22245,22251,22252,22253,22254,22255,22256,22260,22262,22264,22271,22272,22273,22274,22275,22276,22277,22278,22279,22283,22294,22295,22299,22300,22301,22302,22303,22304,22307,22309,22312,22313,22314,

In [ ]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
  
check_id(df_ger,10135)

['COLOURING PENCILS BROWN TUBE']


In [ ]:
frequent_itemsets = apriori(ger_inv_pro_df,
                            min_support=0.01,
                            use_colnames=True)

,support,itemsets
233,0.203666,(22326)
20,0.152749,(20719)
235,0.136456,(22328)
134,0.126273,(21731)
68,0.109980,(21238)
...,...,...
5877,0.010183,"(21577, 21429, 21527)"
5876,0.010183,"(22353, 21481, 21429)"
5874,0.010183,"(21432, 21731, 21429)"
5873,0.010183,"(21432, 21429, 21680)"


In [ ]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

In [ ]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [ ]:
arl_recommender(rules, 22492, 1)

[22636]

In [ ]:
arl_recommender(rules, 22636, 1)

[22328]

In [ ]:
arl_recommender(rules, 22328, 1)

[21398]

In [ ]:
check_id(df_ger,22636)

['CHILDS BREAKFAST SET CIRCUS PARADE']


In [ ]:
check_id(df_ger,22328)

['ROUND SNACK BOXES ,SET 4, FRUITS ']
